In [4]:
import spacy

spacy.require_gpu()

True

In [6]:
!python -m spacy train config.cfg --verbose --output ner_demo/training/  --paths.train train.spacy --paths.dev valid.spacy  --gpu-id 0

[2022-03-05 17:46:41,342] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: ner_demo/training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-03-05 17:46:44,160] [INFO] Set up nlp object from config
[2022-03-05 17:46:44,176] [DEBUG] Loading corpus from path: valid.spacy
[2022-03-05 17:46:44,178] [DEBUG] Loading corpus from path: train.spacy
[2022-03-05 17:46:44,178] [INFO] Pipeline: ['transformer', 'ner']
[2022-03-05 17:46:44,185] [INFO] Created vocabulary
[2022-03-05 17:46:44,186] [INFO] Finished initializing nlp object
Downloading: 100%|██████████████████████████████| 481/481 [00:00<00:00, 341kB/s]
Downloading: 100%|███████████████████████████| 878k/878k [00:00<00:00, 1.42MB/s]
Downloading: 100%|████████████████████████████| 446k/446k [00:00<00:00, 869kB/s]
Downloading: 100%|█████████████████████████| 1.29M/1.29M [00:00<00:00, 2.13MB/s]
Downloading: 100%|███████████████████████████| 478M/4

In [7]:
import pandas as pd
import spacy
test_abstract_df =pd.read_csv('../data/test/abstracts_test.csv', sep='\t')
test_abstract_df.head()

,abstract_id,title,abstract
0,1711760,Delayed institution of hypertension during foc...,The effect of induced hypertension instituted ...
1,6086495,Localisation of the Becker muscular dystrophy ...,A linkage study in 30 Becker muscular dystroph...
2,7018927,Pituitary response to luteinizing hormone-rele...,The effects of a 6-hour infusion with haloperi...
3,7811247,X-linked adrenoleukodystrophy (ALD): a novel m...,Fragments of the adrenoleukodystrophy (ALD) cD...
4,8944024,Detection of heterozygous mutations in BRCA1 u...,The ability to scan a large gene rapidly and a...


In [8]:
ner = spacy.load(R"ner_demo/training/model-best") #load the best model

/opt/conda/lib/python3.8/site-packages/spacy/util.py:1635: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [9]:
# display sentence involving original entities
test_sample =test_abstract_df.iloc[1]['abstract']
spacy.displacy.render(ner(test_sample),jupyter=True, style = "ent")


In [10]:


rec_entity_mentions= []
#test_sentences = [x[0] for x in TEST_DATA[0:4000]] # extract the sentences from [sentence, entity]
for i, row in test_abstract_df.iterrows():
    abs_id = row['abstract_id']
    text = row['title']+' '+row['abstract']
    doc = ner(text)
    for ent in doc.ents:
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        rec_entity_mentions.append([i, abs_id, ent.start_char, ent.end_char,  ent.label_])
    #displacy.render(doc,jupyter=True, style = "ent")

In [29]:
submission_df = pd.DataFrame(rec_entity_mentions, columns=['id','abstract_id','offset_start','offset_finish','type'])

In [30]:
submission_df.to_csv('submission_10.csv', sep='\t', index=False)

In [ ]:
!tar -zcvf ner_demo.tar.gz ner_demo

ner_demo/
ner_demo/training/
ner_demo/training/model-last/
ner_demo/training/model-last/tokenizer
ner_demo/training/model-last/meta.json
ner_demo/training/model-last/config.cfg
ner_demo/training/model-last/transformer/
ner_demo/training/model-last/transformer/cfg
ner_demo/training/model-last/transformer/model
ner_demo/training/model-last/ner/
ner_demo/training/model-last/ner/model
ner_demo/training/model-last/ner/moves
ner_demo/training/model-last/ner/cfg
ner_demo/training/model-last/vocab/
ner_demo/training/model-last/vocab/strings.json
ner_demo/training/model-last/vocab/vectors
ner_demo/training/model-last/vocab/key2row
ner_demo/training/model-last/vocab/vectors.cfg
ner_demo/training/model-last/vocab/lookups.bin
ner_demo/training/model-best/
ner_demo/training/model-best/tokenizer
ner_demo/training/model-best/meta.json
ner_demo/training/model-best/config.cfg
ner_demo/training/model-best/transformer/
ner_demo/training/model-best/transformer/cfg
ner_demo/training/model-best/transformer/

In [11]:
entities_df = pd.read_csv('../data/test/entities_test.csv', sep='\t')
entities_df.head()

,id,abstract_id,offset_start,offset_finish,type,mention,entity_ids
0,0,1711760,23,35,DiseaseOrPhenotypicFeature,hypertension,D006973
1,1,1711760,49,66,DiseaseOrPhenotypicFeature,cerebral ischemia,D002545
2,2,1711760,78,89,DiseaseOrPhenotypicFeature,brain edema,D001929
3,3,1711760,113,125,DiseaseOrPhenotypicFeature,hypertension,D006973
4,4,1711760,165,197,DiseaseOrPhenotypicFeature,middle cerebral artery occlusion,D020244


In [12]:
id2abstr ={ abs_id: abs_text for abs_id, abs_text in test_abstract_df[['abstract_id','abstract']].values }

id2title ={ abs_id: title for abs_id, title in test_abstract_df[['abstract_id','title']].values }

In [13]:
testing_data =[]

for key, df in entities_df.groupby('abstract_id'):
    #print (key)
    entity_mentions = []
    text = id2title[key]+' '+id2abstr[key]
    text = text.lower()
    for i, row in df.iterrows():
          entity_mention = (row['offset_start'], row['offset_finish'],row['type'])
          #print (text[row['offset_start']:row['offset_finish']])
          entity_mentions.append(entity_mention)
    testing_data.append( (text,  {'entities': entity_mentions}))

In [14]:
#import pandas as pd
from tqdm import tqdm
#import spacy
from spacy.tokens import DocBin
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
for text, annot in tqdm(testing_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
            print (start, end, label)
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("test.spacy") # save the docbin object


/opt/conda/lib/python3.8/site-packages/spacy/util.py:1635: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
 49%|████▉     | 49/100 [00:00<00:00, 241.02it/s]

Skipping entity
1285 1289 GeneOrGeneProduct
Skipping entity
607 621 ChemicalEntity
Skipping entity
95 101 SequenceVariant
Skipping entity
583 589 SequenceVariant
Skipping entity
1167 1173 SequenceVariant
Skipping entity
1227 1233 SequenceVariant
Skipping entity
1242 1248 SequenceVariant
Skipping entity
1494 1500 SequenceVariant
Skipping entity
1562 1568 SequenceVariant
Skipping entity
1754 1760 SequenceVariant
Skipping entity
1878 1884 SequenceVariant
Skipping entity
1327 1346 DiseaseOrPhenotypicFeature
Skipping entity
582 590 ChemicalEntity
Skipping entity
593 604 ChemicalEntity
Skipping entity
40 49 SequenceVariant
Skipping entity
345 355 ChemicalEntity
Skipping entity
357 393 ChemicalEntity
Skipping entity
650 654 GeneOrGeneProduct
Skipping entity
694 698 GeneOrGeneProduct
Skipping entity
796 800 GeneOrGeneProduct
Skipping entity
819 823 GeneOrGeneProduct
Skipping entity
1064 1068 GeneOrGeneProduct
Skipping entity
1142 1146 GeneOrGeneProduct
Skipping entity
1190 1194 GeneOrGeneProdu

100%|██████████| 100/100 [00:00<00:00, 252.58it/s]



Skipping entity
568 570 DiseaseOrPhenotypicFeature
Skipping entity
600 602 GeneOrGeneProduct
Skipping entity
638 640 GeneOrGeneProduct
Skipping entity
646 648 DiseaseOrPhenotypicFeature
Skipping entity
650 652 GeneOrGeneProduct
Skipping entity
872 874 GeneOrGeneProduct
Skipping entity
1128 1130 GeneOrGeneProduct
Skipping entity
1024 1032 GeneOrGeneProduct
Skipping entity
1409 1412 GeneOrGeneProduct
Skipping entity
1415 1419 GeneOrGeneProduct
Skipping entity
1422 1427 GeneOrGeneProduct
Skipping entity
1099 1104 GeneOrGeneProduct
Skipping entity
1106 1110 GeneOrGeneProduct
Skipping entity
1714 1718 GeneOrGeneProduct
Skipping entity
1720 1744 GeneOrGeneProduct
Skipping entity
210 213 GeneOrGeneProduct
Skipping entity
215 219 GeneOrGeneProduct
Skipping entity
1170 1173 GeneOrGeneProduct
Skipping entity
1176 1181 GeneOrGeneProduct
Skipping entity
1184 1189 GeneOrGeneProduct
Skipping entity
1081 1086 GeneOrGeneProduct
Skipping entity
1088 1095 GeneOrGeneProduct
Skipping entity
1343 1346 Gen

In [17]:
!spacy evaluate ner_demo/training/model-best test.spacy --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   83.09 
NER R   85.26 
NER F   84.16 
SPEED   16220 


=============================== NER (per type) ===============================

                                 P       R       F
DiseaseOrPhenotypicFeature   81.30   82.83   82.06
ChemicalEntity               81.63   85.39   83.47
OrganismTaxon                93.37   97.51   95.39
GeneOrGeneProduct            84.41   83.83   84.12
SequenceVariant              74.71   87.67   80.67
CellLine                     71.43   58.82   64.52

